In [2]:
import pandas as pd
import numpy as np
from data import DATA_PATH
from sklearn.feature_selection import mutual_info_regression
from source.utils import sarima_fit, find_last_acf_sign_lag, find_last_pacf_sign_lag
def calc_mutual_info(y, X):
    """
    Selects top N features by Mutual Information with target
    """
    # determine the mutual information
    mutual_info = mutual_info_regression(X.fillna(0), y)
    mutual_info = pd.Series(mutual_info)
    mutual_info.index = X.columns
    return list(mutual_info.sort_values(ascending=False).index)


def select_risk_factors(all_data, factor, risk_factors_list, top_factors=4):
    """
    Функция принимает на вход колонку-название актива и список его возможных риск-факторов из словаря
    Отсеиваются факторы с корреляцией между фактором и таргетом < 0.05
    Смотрится взаимная корреляция факторов, если она превышает 0.9, то оставляется один фактор с наибольшей MI с таргетом
    """
    target = all_data[factor].values
    risk_factors = all_data[risk_factors_list]
    drop_list_idx = []
    corr_list = []
    #считаем обычную корреляцию с таргетом
    for i in range(len(risk_factors.columns)):
        corr_with_target = np.corrcoef(x=risk_factors[risk_factors_list[i]].values, y=target)[0][1]
        corr_list.append(corr_with_target)
        if corr_with_target < 0.05:  #дропаем, если меньше 0.05
            drop_list_idx.append(i)

    #ранжируем фичи по MI с таргетом
    top_mutual_information_list = calc_mutual_info(target, risk_factors)

    #смотрим взаимную корреляцию фичей
    mutual_correlation = np.corrcoef(x=risk_factors, rowvar=False)
    a = np.where(mutual_correlation >= 0.9)[0]
    b = np.where(mutual_correlation >= 0.9)[1]
    correlated_features = []
    for i in range(len(a)):
        if a[i] != b[i]:
            correlated_features.append(tuple(sorted([a[i], b[i]])))
    correlated_features = set(correlated_features)  #тут сет пар индексов скоррелированных фичей

    for pair in correlated_features:  #смотрим, какая из фичей из пары на каком месте в ранжированном списке MI
        mi_1 = np.where(np.array(top_mutual_information_list) == risk_factors.columns[pair[0]])
        mi_2 = np.where(np.array(top_mutual_information_list) == risk_factors.columns[pair[1]])

        if mi_1 > mi_2:  #если первый признак из пары менее связан с таргетом (дальше от начала списка MI)
            drop_list_idx.append(pair[0])  #дропаем первый признак
        else:
            drop_list_idx.append(pair[1])

    drop_list_names = list(risk_factors[risk_factors.columns[drop_list_idx]].columns)
    print(drop_list_names)
    return all_data[[factor] + risk_factors_list].drop(drop_list_names, axis=1)
# sarima_fit, find_last_acf_sign_lag, find_last_pacf_sign_lag
# from notebooks.factor_simbaseline import simulation_function


In [3]:
data = pd.read_csv(DATA_PATH / 'all_data.csv')
if 'Unnamed: 0' in data.columns:
    data.drop(columns = ['Unnamed: 0'], inplace = True)
# data.head()

In [6]:
# all_factors_simulated = simulation_function() ## тут в столбцах насимулированные значения факторов, размерность 

In [6]:
from sklearn.linear_model import LinearRegression, Lasso
# 'EUR_RUB', 'USD_RUB'

risk_factors = [
    'su26230_days_before_coupon', 'su26224_days_before_coupon',
    'su26222_days_before_coupon', 'su26221_days_before_coupon',
    'su26218_days_before_coupon', 'ecb_rate', 'aluminum', 'brent',
    'cbr_key_rate', 'eur_rub', 'moex_index', 'nickel', 'rtsi', 'usd_rub', 'pca_cbd' ]

actives_to_predict = ['gazp', 'gmkn',
                      'lkoh', 'magn', 'mgnt', 'moex', 'rosn', 'rual', 'sber', 'vtbr',
                      'su26218', 'su26221', 'su26222', 'su26224', 'su26230']

dict_active_factors = {}

for actives in actives_to_predict:
    dict_active_factors[actives] = select_risk_factors(data, actives, risk_factors_list = risk_factors, top_factors=4).columns

# dict_active_factors




['su26230_days_before_coupon', 'su26222_days_before_coupon', 'su26221_days_before_coupon', 'su26218_days_before_coupon', 'ecb_rate', 'cbr_key_rate', 'eur_rub', 'usd_rub', 'pca_cbd', 'su26221_days_before_coupon', 'usd_rub']
['su26230_days_before_coupon', 'su26224_days_before_coupon', 'su26221_days_before_coupon', 'su26218_days_before_coupon', 'ecb_rate', 'brent', 'cbr_key_rate', 'nickel', 'usd_rub', 'pca_cbd', 'su26221_days_before_coupon', 'eur_rub']
['su26230_days_before_coupon', 'su26224_days_before_coupon', 'su26222_days_before_coupon', 'su26221_days_before_coupon', 'su26218_days_before_coupon', 'brent', 'nickel', 'su26221_days_before_coupon', 'usd_rub']
['su26230_days_before_coupon', 'su26224_days_before_coupon', 'su26222_days_before_coupon', 'su26221_days_before_coupon', 'su26218_days_before_coupon', 'ecb_rate', 'brent', 'cbr_key_rate', 'nickel', 'pca_cbd', 'su26221_days_before_coupon', 'usd_rub']
['su26224_days_before_coupon', 'su26230_days_before_coupon', 'usd_rub']
['su26230_day

In [7]:
dict_active_factors

{'gazp': Index(['gazp', 'su26224_days_before_coupon', 'aluminum', 'brent', 'moex_index',
        'nickel', 'rtsi'],
       dtype='object'),
 'gmkn': Index(['gmkn', 'su26222_days_before_coupon', 'aluminum', 'moex_index', 'rtsi'], dtype='object'),
 'lkoh': Index(['lkoh', 'ecb_rate', 'aluminum', 'cbr_key_rate', 'eur_rub', 'moex_index',
        'rtsi', 'pca_cbd'],
       dtype='object'),
 'magn': Index(['magn', 'aluminum', 'eur_rub', 'moex_index', 'rtsi'], dtype='object'),
 'mgnt': Index(['mgnt', 'su26222_days_before_coupon', 'su26221_days_before_coupon',
        'su26218_days_before_coupon', 'ecb_rate', 'aluminum', 'brent',
        'cbr_key_rate', 'eur_rub', 'moex_index', 'nickel', 'rtsi', 'pca_cbd'],
       dtype='object'),
 'moex': Index(['moex', 'ecb_rate', 'eur_rub', 'moex_index', 'rtsi'], dtype='object'),
 'rosn': Index(['rosn', 'su26222_days_before_coupon', 'su26218_days_before_coupon',
        'ecb_rate', 'aluminum', 'brent', 'cbr_key_rate', 'eur_rub',
        'moex_index', 'rtsi',

In [8]:
def fit_regression(active):
    X = data.loc[:, dict_active_factors[active]]
    y = data[active]
    lr = Lasso().fit(X, y)
    return lr

dict_active_model = {}
for active in actives_to_predict:
    dict_active_model[active] = fit_regression(active)

def predict_simulated(active):
    X = all_factors_simulated.loc[:, dict_active_factors[active]]
    predictions = dict_active_model[active].predict(X)
    return predictions

In [9]:
dict_active_model

{'gazp': Lasso(),
 'gmkn': Lasso(),
 'lkoh': Lasso(),
 'magn': Lasso(),
 'mgnt': Lasso(),
 'moex': Lasso(),
 'rosn': Lasso(),
 'rual': Lasso(),
 'sber': Lasso(),
 'vtbr': Lasso(),
 'su26218': Lasso(),
 'su26221': Lasso(),
 'su26222': Lasso(),
 'su26224': Lasso(),
 'su26230': Lasso()}